# **In this notebook we will filter for HVGs/overdispersed genes for benchmark datasets, and save files for clustering methods**

In [58]:
import h5py
import scanpy as sc
import anndata
import loompy as lp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys


**Download data and metadata for making benchmark loom and h5 files**

In [ ]:
!wget --content-disposition https://data.caltech.edu/records/derx2-tsj62/files/proc_looms_bench.tar.gz?download=1
!tar -xvf proc_looms_bench.tar.gz

### **Save h5 and loom files with HVGs for clustering methods**

In [63]:
def nb_thresh(U,S,var_t = 1.5,u_min =0.02,s_min =0.02):
    '''
    Take in U,S matrices, and find genes that meet var/mean thresh
    U,S are cellxgene
    '''
    var_threshold = var_t
    U_mean = U.mean(0)
    S_mean = S.mean(0)
    U_var = U.var(0)
    S_var = S.var(0)

    #if l == '/home/tchari/counts/allen_bivi/loom/processed_allen_B02H01A02_raw.loom':
    u_min = u_min
    s_min =  s_min


    fitted_idx = (U_mean > u_min) & (S_mean > s_min) \
    & (((U_var-U_mean)/(U_mean**2)) > var_threshold)\
    & (((S_var-S_mean)/(S_mean**2)) > var_threshold)\
    & (np.abs(np.log(S_mean/U_mean)) < 4) 
    
    
    return fitted_idx

In [64]:
#List of looms for benchmark datasets
looms = ['./allen_bivi/loom/processed_allen_A08_raw.loom',
        './allen_bivi/loom/processed_allen_B02H01A02_raw.loom',
        './scMix/cl3/loom/processed_cl3_raw.loom',
        './scMix/cl5/loom/processed_cl5_raw.loom']

short = ['allen_b08','allen_b02h01a02','cl3','cl5']


#Set number of HVGs to try (based on standard procedure)
hvgs = [2000] #[300, 1000, 2000, 4000]

!mkdir ./hvg_objs_0215


for l,s in zip(looms,short):
    ds = lp.connect(l)
    S = ds.layers['spliced'][:,:]
    U = ds.layers['unspliced'][:,:]
    bars = ds.ca['barcode']
    subclass = ds.ca['subclass_label']
    g_names = ds.ra['gene_name']
    ds.close()
    
    
    if l == './allen_bivi/loom/processed_allen_B02H01A02_raw.loom':
        X = U.T.copy() #nuclear data
        print('nuclear')
    else:
        X = S.T.copy()
    
    for h in hvgs:
        
        # --------- Test NB Thresh ----------
        fitted_idx = nb_thresh(U.T,S.T,var_t = 1.5,u_min =0.02,s_min =0.02)
        num_chosen = np.sum(fitted_idx)
        print('No. all genes that pass thresh: ', num_chosen)
        # --------- Test NB Thresh ----------
        
        #Save loom, h5 files with genes that pass NB Thresh
        S_sub = S[fitted_idx,:]
        U_sub = U[fitted_idx,:]
        g_names_sub = g_names[fitted_idx]
        retAdata = anndata.AnnData(
            X=S_sub.T,
            layers={
                'spliced': S_sub.T,
                'unspliced': U_sub.T
            },
            obs=pd.DataFrame({'barcode': bars,'subclass_label':subclass},
                             index=bars),
            var=pd.DataFrame({'gene_name': g_names_sub},index=g_names_sub)
        )

        retAdata.write_loom('./hvg_objs_0215/'+s+'_'+str(num_chosen)+'all.loom')
        
        #h5 files for scMDC
        hf = h5py.File('./hvg_objs_0215/'+s+'_'+str(num_chosen)+'all.h5', 'w')
        hf.create_dataset('X1', data=U_sub.T)
        hf.create_dataset('X2', data=S_sub.T)

        uniqs = dict(zip(np.unique(subclass),list(range(len(np.unique(subclass))))))
        ys = [uniqs[i] for i in subclass]
        hf.create_dataset('Y', data=ys)
        hf.close()
        
        
        # --------- Filter 2k HVGs for genes that pass thresh ----------

        adata = anndata.AnnData(X=X)
        adata.layers["counts"] = adata.X.copy()  # preserve counts
        sc.pp.normalize_total(adata, target_sum=1e4)
        sc.pp.log1p(adata)
        adata.raw = adata
        sc.pp.highly_variable_genes(adata, n_top_genes=h) #min_mean=0.0125, max_mean=3, min_disp=0.25 ?
        #adata = adata[:, adata.var.highly_variable]
        
        
        S_sub = S[adata.var.highly_variable,:]
        U_sub = U[adata.var.highly_variable,:]
        g_names_sub = g_names[adata.var.highly_variable]
        
        #Filtering
        fitted_idx = nb_thresh(U_sub.T,S_sub.T,var_t = 1.5,u_min =0.02,s_min =0.02)
        num_chosen = np.sum(fitted_idx)
        
        print('No. HVGs that pass thresh: ', num_chosen)
        
        S_sub = S_sub[fitted_idx,:]
        U_sub = U_sub[fitted_idx,:]
        g_names_sub = g_names_sub[fitted_idx]
        print(len(g_names_sub))
        
        retAdata = anndata.AnnData(
            X=S_sub.T,
            layers={
                'spliced': S_sub.T,
                'unspliced': U_sub.T
            },
            obs=pd.DataFrame({'barcode': bars,'subclass_label':subclass},
                             index=bars),
            var=pd.DataFrame({'gene_name': g_names_sub},index=g_names_sub)
        )

        retAdata.write_loom('./hvg_objs_0215/'+s+'_'+str(num_chosen)+'hvgs.loom')
        
        #h5 files for scMDC
        hf = h5py.File('./hvg_objs_0215/'+s+'_'+str(num_chosen)+'hvgs.h5', 'w')
        hf.create_dataset('X1', data=U_sub.T)
        hf.create_dataset('X2', data=S_sub.T)

        uniqs = dict(zip(np.unique(subclass),list(range(len(np.unique(subclass))))))
        ys = [uniqs[i] for i in subclass]
        hf.create_dataset('Y', data=ys)
        hf.close()


/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide
/home/tchari/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log


No. all genes that pass thresh:  1948
No. HVGs that pass thresh:  682
682
nuclear
No. all genes that pass thresh:  2770
No. HVGs that pass thresh:  359
359
No. all genes that pass thresh:  1137
No. HVGs that pass thresh:  466
466
No. all genes that pass thresh:  1193
No. HVGs that pass thresh:  357
357


In [65]:
!ls -lh ./hvg_objs_0215
#!rm -r ./hvg_objs

total 814M
-rw-rw-r--. 1 tchari tchari 495M Feb 15 18:46 allen_b02h01a02_2770all.h5
-rw-rw-r--. 1 tchari tchari  45M Feb 15 18:46 allen_b02h01a02_2770all.loom
-rw-rw-r--. 1 tchari tchari  65M Feb 15 18:46 allen_b02h01a02_359hvgs.h5
-rw-rw-r--. 1 tchari tchari  12M Feb 15 18:46 allen_b02h01a02_359hvgs.loom
-rw-rw-r--. 1 tchari tchari  85M Feb 15 18:46 allen_b08_1948all.h5
-rw-rw-r--. 1 tchari tchari  13M Feb 15 18:46 allen_b08_1948all.loom
-rw-rw-r--. 1 tchari tchari  30M Feb 15 18:46 allen_b08_682hvgs.h5
-rw-rw-r--. 1 tchari tchari 6.4M Feb 15 18:46 allen_b08_682hvgs.loom
-rw-rw-r--. 1 tchari tchari 7.9M Feb 15 18:46 cl3_1137all.h5
-rw-rw-r--. 1 tchari tchari 1.4M Feb 15 18:46 cl3_1137all.loom
-rw-rw-r--. 1 tchari tchari 3.3M Feb 15 18:46 cl3_466hvgs.h5
-rw-rw-r--. 1 tchari tchari 869K Feb 15 18:46 cl3_466hvgs.loom
-rw-rw-r--. 1 tchari tchari  36M Feb 15 18:47 cl5_1193all.h5
-rw-rw-r--. 1 tchari tchari 4.6M Feb 15 18:47 cl5_1193all.loom
-rw-rw-r--. 1 tchari tchari  11M Feb 15 18:47 cl5

In [75]:
# !rm ./hvg_objs/meK_looms.tar.gz
# !rm ./hvg_objs/meK_h5s.tar.gz

In [67]:
#Check that counts didn't get transformed (should be discrete)
test = lp.connect('./hvg_objs_0215/allen_b08_682hvgs.loom')
testS = test.layers['unspliced'][:,:]
testS

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0., 24., ...,  1.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [68]:
np.all(testS.astype(int) == testS)

True

In [69]:
test.close()

In [70]:
!tar -cvzf ./hvg_objs_0215/meK_looms.tar.gz ./hvg_objs_0215/*.loom

./hvg_objs_0215/allen_b02h01a02_2770all.loom
./hvg_objs_0215/allen_b02h01a02_359hvgs.loom
./hvg_objs_0215/allen_b08_1948all.loom
./hvg_objs_0215/allen_b08_682hvgs.loom
./hvg_objs_0215/cl3_1137all.loom
./hvg_objs_0215/cl3_466hvgs.loom
./hvg_objs_0215/cl5_1193all.loom
./hvg_objs_0215/cl5_357hvgs.loom


In [71]:
!tar -cvzf ./hvg_objs_0215/meK_h5s.tar.gz ./hvg_objs_0215/*.h5

./hvg_objs_0215/allen_b02h01a02_2770all.h5
./hvg_objs_0215/allen_b02h01a02_359hvgs.h5
./hvg_objs_0215/allen_b08_1948all.h5
./hvg_objs_0215/allen_b08_682hvgs.h5
./hvg_objs_0215/cl3_1137all.h5
./hvg_objs_0215/cl3_466hvgs.h5
./hvg_objs_0215/cl5_1193all.h5
./hvg_objs_0215/cl5_357hvgs.h5
